# Example notebook for 3D lithology

In this example notebook, we show how to quickly create a 3D lithologic bloc of folded layers, uplift it within fastscape and monitor the output.

## Importing the model

The first step is to import the modules related to the model and the visualisation packages. Here we use xsimlab and matplotlib

In [2]:
import fastscape_litho as fstl
import fastscape
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
import zarr
import random
%matplotlib widget
%load_ext xsimlab.ipython

## Preloading a model

We preload the sediment model adapted to have the 3D lithologic block. We also add the Quick Topographic Analysis toolkit.

In [3]:
mymod = fstl.sediment_model_label3D.update_processes({"TA": fstl.QuickTA}) # full model

## Setting up the geometry of the cube

This section sets up the geometry of the 3D matrix. It designs the x,y, and z coordinates as well as the resolution.

In [4]:
# Number of row, cols
ny,nx = 200,200
# length in x and y directions
ly,lx = 5e4,5e4
# Number of discretise boxes in Z
nz = 2000
# Resolution in Z
dz = 10
# Creating an empty 3D matrix and initialising it to 0
zz = np.arange(0,nz*dz,dz) # Intermediate coordinate for later calculations
xz = np.zeros((nz,nx)) # Intermediate coordinate for later calculations
labmat = np.zeros((ny,nx,nz), dtype = np.int8) # Data type np.int8 is an memory optimisation. Each cube takes 1 byte/8 bits of memory (-128 to 127)

## Setting up a "geology"

Bellow, we generate a fake folded landscape. Note that this is for the sake of demonstrating the capacity of the module, and this cross-section is not tuned to be geologically relevant, but "reasonably realistic-looking" (i.e. just for fun). 

We are using a sin wave, with a,b,c,d parameter controlling the width, shift, ... and some thickness param. At some point we will make some parametrisable landscapes for MC analysis of shapes, and some bridges with `gempy` for more realistic cases.

In [6]:
#Setting arbitrary parameters
a = 0.1
b = 100
c = 300
d = 150
# Thickness of each layer **IN NUMBERS OF ROW** 
thisckness = 200
# Building a 2D cross section in X Z coordinate
for i in range(nx):
    tz = d * np.cos(a * (i + b)) + c
    tz = round(tz)
    xz[tz:,i] = 1
    tid = 2
    rand = 0
    for j in range(80):
        tz = d * np.cos(a * (i + b)) + c + thisckness * j
        tz = round(tz)
        xz[tz:,i] = tid
        tid+=1
        if(tid == 3):
            tid = 0
            
#Expanding it through the whole
for i in range(nx):
    labmat[:,i,:] = np.transpose(xz)
    
# Plotting a cross section in XZ to check
fig,ax = plt.subplots(figsize = (6,6))
ax.imshow(labmat[:,10,:], extent = [zz.min(), zz.max(), 0,ly, ], aspect = 'auto', cmap = 'terrain') 
ax.grid()
ax.set_xlabel("Depth (m)")
ax.set_ylabel("X (m)")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'X (m)')

## Setting up Process-specific parameters

We need to give values for the global parameters (e.g. m,n,G,...) and for teh label specific parameters. These are under the form of a 1D array where the indice is the label and the value the param. Note that the last value will be the default one if the block is totally eroded.

In [7]:
# The stream power law exponents
m,n = 0.45,1
# The flow exponent (see https://fastscape.org/fastscapelib-fortran/#_fastscape_set_erosional_parameters parameter p for explanations)
flowexp = 7
# Fluvial erodibility of sediments
Ksoil = 5e-4
# Fluvial deposition
G_r = 0.1
G_s = 0.2

# Uplift field 
Up = np.full( (ny,nx) , 1e-3 )

# Array of rock-specific params:
## Bedrock fluvial K
Krs = np.array([1e-4,2e-4, 0.6e-4, 1e-4]) * 0.2
## Bedrock Hillslope K
Kdrs = np.array([1e-1,2e-1, 0.6e-1,1e-1]) * 2
## Sediment Hillslope K
Kdss = Kdrs * 1.3 * 0.2

# Timy wimy stuff
dt = 1e4 # time step of 10,000 years
time = np.arange(0,2e7,dt) # Running for 20 millions years
otime = time[::5] # outputting every 5 time steps

## Setting up the Xsimlab fastscape model
See xsimlab documetnation for details. Here I am outputting the elevation, erosion field, indicies at surface (= "geological map"), drainage area, ksn, and few drainage divide monitoring indices.

In [8]:
# %create_setup mymod
import xsimlab as xs

ds_in = xs.create_setup(
    model=mymod,
        clocks={
        'time':time,
        'otime':otime,
    },
    master_clock= 'time',
    input_vars={
        # The grid extent in number of cells 
        'grid__shape': [ny,nx],
        # the grid length in meter (full length)
        'grid__length': [ly,lx],
        #Boundary status EWNS. Looped = periodic
        'boundary__status': ['looped', 'looped', 'fixed_value', 'fixed_value'],
        # Uplift rate
        'uplift__rate': Up,
        # Random seed for terrain generation (-> a constant number + grid shape will always produce the same initial terrain)
        'init_topography__seed': 21,
        # Exponent for Multiple Flow (/!\ carefull, the TA based on chi are not really accurate with Multiple Flow)
        'flow__slope_exp': flowexp,
        # m exponent of the SPL
        'spl__area_exp': m,
        # n exponent of the SPL
        'spl__slope_exp': n,
        #K for soft sediments
        'spl__k_coef_soil': Ksoil,
        #K G for bedrock
        'spl__g_coef_bedrock': G_r,
        #K G for soil
        'spl__g_coef_soil': G_s,
        # Depth resolution
        'label__dz': dz,
        # Depth origin
        'label__origin_z': 0,
        # 3D matrice of geology
        'label__labelmatrix': labmat,
        # label2K4bedrock
        'spl__Kr_lab': Krs,
        # label2K4hillslope (soil)
        'diffusion__Kdr_lab': Kdrs,
        # label2K4hillslope (bedrock)
        'diffusion__Kds_lab': Kdss, 
        # Theta_ref for topographic analysis
        'TA__theta_chi': m/n,
        # !_0 for chi extraction 
        'TA__A_0_chi': 1,
        # Threshold for river extraction
        'TA__minAcc': 1e4,
        # Path for saving river profile data (/!\ The folder need to exists!)
        'TA__output_prefix': './test_output_csv/Folded',
        # Specific parameter for drainage divide analysis
        'TA__main_divide_contrast_distance': 2000,
    },
    output_vars={
        # Topo
        'topography__elevation': 'otime',
        # A
        'drainage__area' : 'otime',
        # "geological map"
        'label__indices': 'otime',
        # k_sn
        'TA__ksnSA': 'otime',
        # E
        'erosion__rate' : 'otime',
        # Rough proxy for drainage divide migration rate
        'TA__main_drainage_divides_migration_index': 'otime',
        # Centrail main divide
        'TA__main_divide': 'otime',
        
    }
)

## Running the model

As the title suggests, runs the model

In [9]:
with mymod, xs.monitoring.ProgressBar():
#     out_ds = ds_in.xsimlab.run(store=zarr.TempStore())
    out_ds = ds_in.xsimlab.run()

             0% | initialize 

## Using `ipyfastscape` to monitor the output in 3D

This package visualises the thingy in 3D. See [here](https://github.com/fastscape-lem/ipyfastscape) for details about installation

In [10]:
from ipyfastscape import TopoViz3d,AppLinker
app2 = TopoViz3d(out_ds.load(), canvas_height=600, time_dim='otime')
app2.show()


Output(layout=Layout(height='640px'))

## Few quick figures to monitor the Drainage Divide and other outputs

Basically just using few of the model capacities to extract some metrics! 

### First a figure showing in red the median/quartiles erosion rate and in gray the "naive" extraction of $k_{sn}$


In [11]:
X = out_ds['otime'].values
Y = []
Y_1st = []
Y_3rd = []
ErrY = []
ErrY_1st = []
ErrY_3rd = []
for i in range(out_ds.otime.values.shape[0]):
    Y.append(np.median(out_ds["TA__ksnSA"].values[i][out_ds["TA__ksnSA"].values[i]>0]))
    Y_1st.append(np.percentile(out_ds["TA__ksnSA"].values[i][out_ds["TA__ksnSA"].values[i]>0], 25))
    Y_3rd.append(np.percentile(out_ds["TA__ksnSA"].values[i][out_ds["TA__ksnSA"].values[i]>0], 75))
    ErrY.append(np.median(out_ds["erosion__rate"].values[i][out_ds["TA__ksnSA"].values[i]>0]))
    ErrY_1st.append(np.percentile(out_ds["erosion__rate"].values[i][out_ds["TA__ksnSA"].values[i]>0], 25))
    ErrY_3rd.append(np.percentile(out_ds["erosion__rate"].values[i][out_ds["TA__ksnSA"].values[i]>0], 75))


fig, ax = plt.subplots(figsize = (7,6))
ax.set_facecolor('k')
ax.plot(X,Y, color = 'w', lw = 1.5)
ax.fill_between(X,Y_1st,Y_3rd, color = 'w', alpha = 0.5, lw = 0)


ax2 = ax.twinx()
ax2.plot(X,ErrY, color = 'r', lw = 1.5)
ax2.fill_between(X,ErrY_1st,ErrY_3rd, color = 'r', alpha = 0.5, lw = 0)
ax2.set_ylim(0.1e-3,2e-3)
ax.set_ylim(0,80)
ax2.set_ylim(0.75e-3,1.25e-3)
ax.grid(ls = '--')

ax.set_xlabel("Time (yrs)")
ax.set_ylabel(r"$k_{sn}$ ($m^{2\theta}$)")
ax2.set_ylabel(r"Erosion rate $m.yrs^{-1}$")
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Then an approximation of drainage divide migration (Number of pixel becoming drainage divide compared to previous step)

In [12]:
import scipy.stats as stats
import helplotlib as hpl

step = 1e5
fig, ax = hpl.mkfig_grey_bold(figsize = (6,5))

ax.scatter(out_ds.otime.values[:], out_ds.TA__main_drainage_divides_migration_index.values,lw = 0, c = "grey",s = 4)

X = np.arange(out_ds.otime.values.min(), out_ds.otime.values.max(), step)
migID = out_ds.TA__main_drainage_divides_migration_index.values
migID[np.isnan(migID)] = 0
Y = stats.binned_statistic(out_ds.otime.values, out_ds.TA__main_drainage_divides_migration_index.values, statistic = np.median, bins=X)[0]
ax.plot(X[:-1]/2 + X[1:] / 2,Y, c = 'k', lw = 2)

ax.set_ylim(10,125)
ax.set_xlabel("Time (years)")
ax.set_ylabel("N pixels migrating")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'N pixels migrating')

### Finally, Sediment fluxes escaping the model from the North, vs from the South

In [20]:
flux_N = []
flux_S = []
cellarea = ly/(ny-1) * lx/(nx-1)
for t in out_ds.otime.values:
    mask_N = out_ds["TA__main_divide"].sel({'otime': t}).values == 3
    flux_N.append(out_ds["erosion__rate"].sel({'otime': t}).values[mask_N])
    flux_S.append(out_ds["erosion__rate"].sel({'otime': t}).values[mask_N==False])
    flux_N[-1] = np.nansum(flux_N[-1]) * cellarea
    flux_S[-1] = np.nansum(flux_S[-1]) * cellarea
#     print(flux_N[-1].shape[0],flux_S[-1].shape[0])
#     print(flux_N[-1],flux_S[-1])

    
fig, ax = hpl.mkfig_grey_bold(figsize = (6,5))
ax.plot(out_ds.otime.values,flux_N, color = "red", label = "N")
ax.plot(out_ds.otime.values,flux_S, color = "blue", label = "S")
ax.set_xlabel("Time (years)")
ax.set_ylabel("$Q_s^{out}$ in $m^3$")
# ax.set_yscale('log')
ax.legend()
ax.set_ylim(9e5,1.6e6)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(900000.0, 1600000.0)